# Analysis of Load Peakiness by Balancing Authority

This notebook analyzes changes in the peak end of the load divergence curve by Balancing Authority.

In [ ]:
# Start by importing the packages we need:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from glob import glob


## Set the Directory Structure

In [ ]:
# Identify the top-level directory and the subdirectory where the data will be stored:
data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output/'
data_output_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2023_applied_energy/data/peakiness/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2023_applied_energy/figures/'


## Process the Annual Mean Load Difference

In [ ]:
# Define a function to calculate the annual load duration curve:
def process_ba_peakiness(data_input_dir: str, data_output_dir: str, threshold: int):
    # Initiate a counter to store the results:
    counter = 0;
    output_df = pd.DataFrame()
    
    # Set the scenarios to process:
    scenarios = ['historic', 'rcp45cooler_ssp3', 'rcp45cooler_ssp5', 'rcp45hotter_ssp3', 'rcp45hotter_ssp5', 'rcp85cooler_ssp3', 'rcp85cooler_ssp5', 'rcp85hotter_ssp3', 'rcp85hotter_ssp5']

    # Loop over the scenarios and process each time series:
    for s in range(len(scenarios)):
        if scenarios[s] == 'historic':
           start_year = 1980; end_year = 2020; interval = 1;
        else:
           start_year = 2020; end_year = 2100; interval = 5;
        
        # Loop over the years from the start_year to the end_year:
        for year in range(start_year, end_year, interval):
            # Read in the TELL BA output file for that year and scenario:
            tell_df = pd.read_csv(data_input_dir + scenarios[s] + '/' + str(year) + '/TELL_Balancing_Authority_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv')
             
            # Make a list of all of the BAs:
            bas = tell_df['BA_Code'].unique()
         
            # Loop over the BAs and calculate peak load statistics for each year and BA combination:
            for i in range(len(bas)):
                # Subset to just the data for the BA being processed:
                subset_df = tell_df[tell_df['BA_Code'].isin([bas[i]])].copy()
                
                # Calculate the load normalized by the annual maximum value:
                subset_df['Load_Normalized'] = (subset_df['Scaled_TELL_BA_Load_MWh'] / subset_df['Scaled_TELL_BA_Load_MWh'].max())
                
                # Iterate the counter by one:
                counter = counter + 1
                
                # Put the output in a new dataframe:
                output_df.loc[counter, 'Year'] = str(year)
                output_df.loc[counter, 'BA'] = bas[i]
                output_df.loc[counter, 'Scenario'] = scenarios[s]
                output_df.loc[counter, 'Threshold'] = threshold
                output_df.loc[counter, 'Peak_Load'] = subset_df['Scaled_TELL_BA_Load_MWh'].max()
                output_df.loc[counter, 'Hours_Above_Threshold'] = len(subset_df[subset_df['Load_Normalized'] >= threshold])
                output_df.loc[counter, 'Percent_Above_Threshold'] = 100*(len(subset_df[subset_df['Load_Normalized'] >= threshold]) / len(subset_df))

                # Clean up and move to the next step in the loop:
                del subset_df
    
    # Generate the .csv output file name:
    csv_output_filename = os.path.join(data_output_dir, 'BA_Peakiness_' + str(threshold) + '_Threshold.csv')
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv(csv_output_filename, sep=',', index=False)
    
    return output_df


In [ ]:
# Process the BA peakiness dataframe:
output_df = process_ba_peakiness(data_input_dir = data_input_dir,
                                 data_output_dir = data_output_dir,
                                 threshold = 0.9)

output_df


In [ ]:
# Define a function to calculate the change in peakiness:
def process_ba_peakiness_deltas(data_input_dir: str, data_output_dir: str, threshold: int):
    # Initiate a counter to store the results:
    counter = 0;
    output_df = pd.DataFrame()
    
    # Check to see if the processed output file exist and if not then create it:
    if os.path.isfile(os.path.join(data_output_dir, 'BA_Peakiness_' + str(threshold) + '_Threshold.csv')) == False:
       # Process the BA peakiness dataframe:
       peak_df = process_ba_peakiness(data_input_dir = data_input_dir,
                                        data_output_dir = data_output_dir,
                                        threshold = 0.9) 
    else:
       # Read in the output file:
       peak_df = pd.read_csv(os.path.join(data_output_dir, 'BA_Peakiness_' + str(threshold) + '_Threshold.csv'))
    
    # Make a list of all of the BAs:
    bas = peak_df['BA'].unique()
         
    # Loop over the BAs and calculate peak load statistics for each year and BA combination:
    for i in range(len(bas)):
    #for i in range(1):
        # Subset to just the data for the BA being processed:
        subset_df = peak_df[peak_df['BA'].isin([bas[i]])].copy()
        
        # Set the scenarios to process:
        scenarios = ['rcp45cooler_ssp3', 'rcp45cooler_ssp5', 'rcp45hotter_ssp3', 'rcp45hotter_ssp5', 'rcp85cooler_ssp3', 'rcp85cooler_ssp5', 'rcp85hotter_ssp3', 'rcp85hotter_ssp5']
        
        # Loop over the scenarios and process each time series:
        for s in range(len(scenarios)):
            
            # Loop over the years from the start_year to the end_year:
            for year in range(2020, 2100, 5):
                
                # Set the time change value:
                if year < 2060:
                   year_delta = -40
                else:
                   year_delta = -80
                
                # Iterate the counter by one:
                counter = counter + 1
                
                # Put the output in a new dataframe:
                output_df.loc[counter, 'Year'] = year
                output_df.loc[counter, 'BA'] = bas[i]
                output_df.loc[counter, 'Scenario'] = scenarios[s]
                output_df.loc[counter, 'Threshold'] = threshold
                output_df.loc[counter, 'Future_Value'] = subset_df['Hours_Above_Threshold'].loc[(subset_df['Year'] == year) & (subset_df['Scenario'] == scenarios[s])].values
                output_df.loc[counter, 'Historical_Value'] = subset_df['Hours_Above_Threshold'].loc[(subset_df['Year'] == (year + year_delta)) & (subset_df['Scenario'] == 'historic')].values
                output_df.loc[counter, 'Delta'] = output_df.loc[counter,'Future_Value'] - output_df.loc[counter,'Historical_Value']
        
        del subset_df, scenarios, s, year
    
    # Break the data down into the two era:
    era_one = output_df.loc[(output_df['Year'] <= 2059)].copy()
    era_two = output_df.loc[(output_df['Year'] >= 2060)].copy()
    
    # Sum the county loads as a function of time:
    era_one['Era_One_Delta_Value'] = era_one.groupby(['BA','Scenario'])['Delta'].transform('mean')
    era_two['Era_Two_Delta_Value'] = era_two.groupby(['BA','Scenario'])['Delta'].transform('mean')
    
    # Merge the two stats dataframes together and output the combined data:
    stats_one_df = era_one[['BA', 'Scenario', 'Era_One_Delta_Value']].drop_duplicates()
    stats_two_df = era_two[['BA', 'Scenario', 'Era_Two_Delta_Value']].drop_duplicates()
    
    stats_df = stats_one_df.merge(stats_two_df, on=['BA', 'Scenario'])
    
    # Generate the .csv output file name:
    csv_output_filename = os.path.join(data_output_dir, 'BA_Peakiness_Statistics_' + str(threshold) + '_Threshold.csv')
    
    # Write out the dataframe to a .csv file:
    stats_df.to_csv(csv_output_filename, sep=',', index=False)
    
    return stats_df


In [ ]:
# Process the change in peakiness:
stats_df = process_ba_peakiness_deltas(data_input_dir = data_input_dir,
                                       data_output_dir = data_output_dir,
                                       threshold = 0.9)

stats_df


## Make the Plots

In [ ]:
# Define a function to plot the time series of annual demand for each BA:
def plot_ba_peakiness(ba_to_plot: str, data_input_dir: str, data_output_dir: str, threshold: int, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Check to see if the processed output file exist and if not then create it:
    if os.path.isfile(os.path.join(data_output_dir, 'BA_Peakiness_' + str(threshold) + '_Threshold.csv')) == False:
       # Process the BA peakiness dataframe:
       output_df = process_ba_peakiness(data_input_dir = data_input_dir,
                                        data_output_dir = data_output_dir,
                                        threshold = threshold) 
    else:
       # Read in the output file:
       output_df = pd.read_csv(os.path.join(data_output_dir, 'BA_Peakiness_' + str(threshold) + '_Threshold.csv')) 
    
    # Subset to just the BA you want to plot:
    ba_df = output_df[output_df['BA'] == ba_to_plot]
    
    # Subset to the time series for each scenario:
    ba_historic = ba_df[ba_df['Scenario'].isin(['historic'])]
    ba_rcp45cooler_ssp3 = ba_df[ba_df['Scenario'].isin(['rcp45cooler_ssp3'])]
    ba_rcp45cooler_ssp5 = ba_df[ba_df['Scenario'].isin(['rcp45cooler_ssp5'])]
    ba_rcp45hotter_ssp3 = ba_df[ba_df['Scenario'].isin(['rcp45hotter_ssp3'])]
    ba_rcp45hotter_ssp5 = ba_df[ba_df['Scenario'].isin(['rcp45hotter_ssp5'])]
    ba_rcp85cooler_ssp3 = ba_df[ba_df['Scenario'].isin(['rcp85cooler_ssp3'])]
    ba_rcp85cooler_ssp5 = ba_df[ba_df['Scenario'].isin(['rcp85cooler_ssp5'])]
    ba_rcp85hotter_ssp3 = ba_df[ba_df['Scenario'].isin(['rcp85hotter_ssp3'])]
    ba_rcp85hotter_ssp5 = ba_df[ba_df['Scenario'].isin(['rcp85hotter_ssp5'])]
    
    # Subset to just the 5-year intervals:
    ba_historic = ba_historic[ba_historic['Year'].isin({1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015})]
    historical_max = ba_historic['Hours_Above_Threshold'].max()
    historical_min = ba_historic['Hours_Above_Threshold'].min()
    
    # Make the raw load plots:
    plt.figure(figsize=(24, 12))
    plt.rcParams['font.size'] = 16
    
    plt.plot(ba_historic['Year'], ba_historic['Hours_Above_Threshold'], color='black', linestyle='-', label='historic', linewidth=2)
    plt.plot(ba_rcp45cooler_ssp3['Year'], ba_rcp45cooler_ssp3['Hours_Above_Threshold'], color='blue', linestyle=':', label='rcp45cooler_ssp3', linewidth=2)
    plt.plot(ba_rcp45cooler_ssp5['Year'], ba_rcp45cooler_ssp5['Hours_Above_Threshold'], color='blue', linestyle='-', label='rcp45cooler_ssp5', linewidth=2)
    plt.plot(ba_rcp45hotter_ssp3['Year'], ba_rcp45hotter_ssp3['Hours_Above_Threshold'], color='cyan', linestyle=':', label='rcp45hotter_ssp3', linewidth=2)
    plt.plot(ba_rcp45hotter_ssp5['Year'], ba_rcp45hotter_ssp5['Hours_Above_Threshold'], color='cyan', linestyle='-', label='rcp45hotter_ssp5', linewidth=2)
    plt.plot(ba_rcp85cooler_ssp3['Year'], ba_rcp85cooler_ssp3['Hours_Above_Threshold'], color='orange', linestyle=':', label='rcp85cooler_ssp3', linewidth=2)
    plt.plot(ba_rcp85cooler_ssp5['Year'], ba_rcp85cooler_ssp5['Hours_Above_Threshold'], color='orange', linestyle='-', label='rcp85cooler_ssp5', linewidth=2)
    plt.plot(ba_rcp85hotter_ssp3['Year'], ba_rcp85hotter_ssp3['Hours_Above_Threshold'], color='red', linestyle=':', label='rcp85hotter_ssp3', linewidth=2)
    plt.plot(ba_rcp85hotter_ssp5['Year'], ba_rcp85hotter_ssp5['Hours_Above_Threshold'], color='red', linestyle='-', label='rcp85hotter_ssp5', linewidth=2)
    plt.plot([1980, 2100], [historical_min, historical_min], color='gray', linestyle='--', label='', linewidth=2)
    plt.plot([1980, 2100], [historical_max, historical_max], color='gray', linestyle='--', label='', linewidth=2)
    
    plt.xlim([1980, 2100]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100],['1980','','2000','','2020','','2040','','2060','','2080','','2100'])
    plt.legend(loc='upper left', prop={'size': 12})
    plt.ylabel('Hours Above 90% of the Annual Maximum Load')
    plt.grid(False)
    plt.title((ba_to_plot + ' Peakiness Projections'))
    

In [ ]:
output_df = plot_ba_peakiness(ba_to_plot = 'CISO',
                              data_input_dir = data_input_dir,
                              data_output_dir = data_output_dir,
                              threshold = 0.9,
                              image_output_dir = image_output_dir, 
                              image_resolution = 50, 
                              save_images = False)

output_df


In [ ]:
# Define a function to plot change in peakiness across BAs:
def plot_ba_peakiness_deltas(scenario_to_plot: str, data_input_dir: str, data_output_dir: str, threshold: int, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Check to see if the processed output file exist and if not then create it:
    if os.path.isfile(os.path.join(data_output_dir, 'BA_Peakiness_Statistics_' + str(threshold) + '_Threshold.csv')) == False:
       # Process the BA peakiness dataframe:
       stats_df = process_ba_peakiness_deltas(data_input_dir = data_input_dir,
                                              data_output_dir = data_output_dir,
                                              threshold = threshold)
    else:
       # Read in the output file:
       stats_df = pd.read_csv(os.path.join(data_output_dir, 'BA_Peakiness_Statistics_' + str(threshold) + '_Threshold.csv'))
    
    # Subset to just the scenario you want to plot:
    plot_df = stats_df[stats_df['Scenario'] == scenario_to_plot]
    
    # Create an x-axis the length of the dataframe to be used in plotting:
    x_axis = np.arange(len(plot_df))
    
    # Make the plot:
    plt.figure(figsize=(24, 11))
    plt.rcParams['font.size'] = 14
    
    plt.subplot(211)
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp45cooler_ssp5')], stats_df['Era_One_Delta_Value'].loc[(stats_df['Scenario']=='rcp45cooler_ssp5')], s=75, c='blue', label='rcp45cooler_ssp5')
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp45hotter_ssp5')], stats_df['Era_One_Delta_Value'].loc[(stats_df['Scenario']=='rcp45hotter_ssp5')], s=75, c='cyan', label='rcp45hotter_ssp5')
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp85cooler_ssp5')], stats_df['Era_One_Delta_Value'].loc[(stats_df['Scenario']=='rcp85cooler_ssp5')], s=75, c='orange', label='rcp85cooler_ssp5')
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp85hotter_ssp5')], stats_df['Era_One_Delta_Value'].loc[(stats_df['Scenario']=='rcp85hotter_ssp5')], s=75, c='red', label='rcp85hotter_ssp5')
    plt.grid()
    plt.gca().set_axisbelow(True)
    plt.xticks(color='w')
    plt.xlim(-1, 54)
    plt.ylim(-165, 165)
    plt.xticks(rotation=90)
    plt.xlabel('')
    plt.ylabel('Delta Hours > 90% of the Annual Max')
    plt.title('Annual Mean Change in Load Peakiness: 2020-2059')
    plt.title('a)', loc='left', fontsize=13)
    
    plt.subplot(212)
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp45cooler_ssp5')], stats_df['Era_Two_Delta_Value'].loc[(stats_df['Scenario']=='rcp45cooler_ssp5')], s=75, c='blue', label='rcp45cooler_ssp5')
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp45hotter_ssp5')], stats_df['Era_Two_Delta_Value'].loc[(stats_df['Scenario']=='rcp45hotter_ssp5')], s=75, c='cyan', label='rcp45hotter_ssp5')
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp85cooler_ssp5')], stats_df['Era_Two_Delta_Value'].loc[(stats_df['Scenario']=='rcp85cooler_ssp5')], s=75, c='orange', label='rcp85cooler_ssp5')
    plt.scatter(stats_df['BA'].loc[(stats_df['Scenario']=='rcp85hotter_ssp5')], stats_df['Era_Two_Delta_Value'].loc[(stats_df['Scenario']=='rcp85hotter_ssp5')], s=75, c='red', label='rcp85hotter_ssp5')
    plt.grid()
    plt.gca().set_axisbelow(True)
    plt.legend(loc='lower left', prop={'size': 12})
    plt.xlim(-1, 54)
    plt.ylim(-165, 165)
    plt.xticks(rotation=90)
    plt.xlabel('Balancing Authority')
    plt.ylabel('Delta Hours > 90% of the Annual Max')
    plt.title('Annual Mean Change in Load Peakiness: 2060-2099')
    plt.title('b)', loc='left', fontsize=13)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('Peakiness.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [ ]:
plot_ba_peakiness_deltas(scenario_to_plot = 'rcp85hotter_ssp5', 
                         data_input_dir = data_input_dir,
                         data_output_dir = data_output_dir,
                         threshold = 0.90,
                         image_output_dir = image_output_dir, 
                         image_resolution = 300, 
                         save_images = True)
